In [1]:
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import sqlite3
import mysql.connector as cnt
import random

In [7]:
opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=opcijos)
url = 'https://www.aruodas.lt/butai/puslapis/1/?FOwnerDbId0=1&FOwnerDbId2=1'
driver.get(url)
time.sleep(5)
source = driver.page_source


page = 1
# page = 149
# while page < 201:
while page < 330:
    source = driver.page_source
    
    bs = BeautifulSoup(source, 'html.parser')

    butai_nuorodos = []

    linkai = bs.find_all('h3')
    for linkas in linkai:
        l = linkas.find('a')
        if l != None:
            butai_nuorodos.append(l['href'])

    kitas = bs.find('div', {'class':'pagination'}).find_all('a', {'class':'page-bt'})
    for k in kitas:
        # print(k.text.strip())
        if k.text.strip() == '»':
            kitas_psl = k['href']
            # print(kitas_psl)
    next_nuoroda = f'https://www.aruodas.lt/{kitas_psl}'
    
    for nuoroda in butai_nuorodos:
        # print(nuoroda)
        
        duomenys = {
            'miestas':'None',
            'rajonas':'None',
            'gatve':'None',
            'kaina':'None',
            'kv_kaina':'None',
            'Plotas:':'None',
            'Kambarių sk.:':'None',
            'Metai:':'None',
            'Aukštas:':'None',
            'Aukštų sk.:':'None',
            'Namo numeris:':'None',
            'Buto numeris:':'None',
            'Pastato tipas:':'None',
            'Šildymas:':'None',
            'Įrengimas:':'None',
            'Pastato energijos suvartojimo klasė:':'None',
            'Ypatybės:':'None',
            'Papildomos patalpos:':'None',
            'Papildoma įranga:':'None',
            'Apsauga:':'None',
            'Unikalus daikto numeris (RC numeris):': 'None',
            'nuoroda':'None'
        }

        duomenys['nuoroda'] = nuoroda
        
        url = nuoroda
        driver.get(url)
        time.sleep(3)
        source = driver.page_source

        bs_buto = BeautifulSoup(source, 'html.parser')

        gyvenviete = bs_buto.find('h1', {'class':'obj-header-text'}).text
        gyvenviete_lst = gyvenviete.split(',')
        if len(gyvenviete_lst) == 4:
            miestas = gyvenviete_lst[0].strip()
            duomenys['miestas'] = miestas
            # print(miestas)
            rajonas = gyvenviete_lst[1].strip()
            duomenys['rajonas'] = rajonas
            # print(rajonas)
            gatve = gyvenviete_lst[2].strip()
            duomenys['gatve'] = gatve
            # print(gatve)
        if len(gyvenviete_lst) == 3:
            miestas = gyvenviete_lst[0].strip()
            duomenys['miestas'] = miestas
            # print(miestas)
            gatve = gyvenviete_lst[1].strip()
            duomenys['gatve'] = gatve

            
        kaina = int(bs_buto.find('span', {'class':'price-eur'}).text.strip().replace('€', '').replace(' ', ''))
        duomenys['kaina'] = kaina
        # print(kaina)

        kv_kaina_txt = bs_buto.find('span', {'class':'price-per'}).text.strip()
        # print(kv_kaina_txt)
        kv_kaina = int(kv_kaina_txt.split('€')[0].replace(' ', '').replace('(', ''))
        duomenys['kv_kaina'] = kv_kaina
        # print(kv_kaina)


        info_keys = []
        info_values = []
        info_k = bs_buto.find('dl', {'class':'obj-details'}).find_all('dt')
        for i in info_k:
            # print(i.text.strip())
            info_keys.append(i.text.strip())
            
        # print(info_keys)

        info_v = bs_buto.find('dl', {'class':'obj-details'}).find_all('dd')
        for i in info_v:
            # print(i.text.strip())
            info_values.append(i.text.strip())
        # print(info_values)

        for k, v in zip(info_keys, info_values):
            duomenys[k] = v

        print(duomenys)
        
        data = tuple(duomenys.values())
        data1 = []
        data1.append(data)
        
        SDB = sqlite3.connect('Aruodas_test.db')  # jei neegzistuoja db , bus sukurta nauja db
        Cs = SDB.cursor()
        
        #  if not exists - tikrina ar jau sukurta DB
        sql = '''create table if not exists Aruodas
        (
        miestas text not null,
        rajonas text not null,
        gatve text not null,
        kaina numeric not null,
        kvadatoKaina numeric not null,
        plotas text not null,
        kambariai text not null,
        metai text not null,
        aukstas text not null,
        aukstuSk text not null,
        namoNr text not null,
        butoNr text not null,
        pastatoTipas text not null,
        sildymas text not null,
        irengimas text not null,
        energetineKlase text not null,
        ypatybes text not null,
        papildomosPatalpos text not null,
        papildomaIranga text not null,
        apsauga text not null,
        unikalusNr text not null,
        nuoroda text not null
        )
        '''
        Cs.execute(sql)

        sql_template = '''insert into Aruodas values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'''
        Cs.executemany(sql_template, data1)

        SDB.commit() # commit() butinas jei norim irasyti duomenis i DB

        SDB.close()
        print('data written to DB')
        
        
    driver.get(next_nuoroda)
    
    # su get metodu atsidarom next_nuoroda
    # nuskaitome next puslapi
    # pasiimam nuajas auto nuorodas ir nauja next nuorodas
    # cikla kartojam is naujo
    page = page +1
    print('data written to DB')
    print(f'kitas puslapis: {kitas_psl}')
    
    
driver.close()
print('Baigta!!!')

{'miestas': 'Vilnius', 'rajonas': 'Šnipiškės', 'gatve': 'Verkių g.', 'kaina': 205000, 'kv_kaina': 3076, 'Plotas:': '66,64 m²', 'Kambarių sk.:': '2', 'Metai:': '2003', 'Aukštas:': '3', 'Aukštų sk.:': '5', 'Namo numeris:': '5A', 'Buto numeris:': '55', 'Pastato tipas:': 'Mūrinis', 'Šildymas:': 'Centrinis', 'Įrengimas:': 'Įrengtas', 'Pastato energijos suvartojimo klasė:': 'A', 'Ypatybės:': 'Tualetas ir vonia atskirai\nInternetas\nVirtuvė sujungta su kambariu', 'Papildomos patalpos:': 'Balkonas\nVieta automobiliui', 'Papildoma įranga:': 'Skalbimo mašina\nSu baldais\nŠaldytuvas\nVirtuvės komplektas\nViryklė\nIndaplovė\nVonia', 'Apsauga:': 'Kodinė laiptinės spyna', 'Unikalus daikto numeris (RC numeris):': '1097-1012-6150', 'nuoroda': 'https://www.aruodas.lt/butai-vilniuje-snipiskese-verkiu-g-parduodamas-jaukus-ir-sviesus-butas-butas-1-3416089/'}
data written to DB
{'miestas': 'Palanga', 'rajonas': 'Palanga', 'gatve': 'A. Mickevičiaus g.', 'kaina': 268000, 'kv_kaina': 7657, 'Plotas:': '35 m²',

ProgrammingError: Incorrect number of bindings supplied. The current statement uses 22, and there are 23 supplied.

In [ ]:
def getColNamesList(table, dbname):
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''select * from {table} limit 0'''.format(table=table)
    s = c.execute(sql)
    # r = s.description
    column_names = [description[0] for description in c.description]
    return column_names

def alterTableAddColumn(cols_i_want_to_have, cols_i_have, table, base):
    db = sqlite3.connect(base)
    c = db.cursor()
    sql = '''alter table {tname}
    add column `{n}`
    '''
    for cl in cols_i_want_to_have:
        if cl not in cols_i_have:
            c.execute(sql.format(tname=table, n=cl))
            time.sleep(1)
            db.commit()
            time.sleep(3)
    c.close()
    db.close()
    return True


def createMainTable(dbname:str) -> bool:
    sql = '''
    drop table if exists Aruodas
    '''
    db = sqlite3.connect(dbname)
    c = db.cursor()
    c.execute(sql)
    sql = '''create table if not exists Aruodas
    (
        miestas text not null,
        rajonas text not null,
        gatve text not null,
        kaina numeric not null,
        kvadatoKaina numeric not null
    )
    '''
    c.execute(sql)
    c.close()
    db.close()
    return True
    pass


opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=opcijos)
url = 'https://www.aruodas.lt/butai/puslapis/1/?FOwnerDbId0=1&FOwnerDbId2=1'
driver.get(url)
time.sleep(5)
source = driver.page_source

page = 1
# page = 149
# while page < 201:
while page < 330:
    source = driver.page_source
    
    bs = BeautifulSoup(source, 'html.parser')

    butai_nuorodos = []

    linkai = bs.find_all('h3')
    for linkas in linkai:
        l = linkas.find('a')
        if l != None:
            butai_nuorodos.append(l['href'])

    kitas = bs.find('div', {'class':'pagination'}).find_all('a', {'class':'page-bt'})
    for k in kitas:
        # print(k.text.strip())
        if k.text.strip() == '»':
            kitas_psl = k['href']
            # print(kitas_psl)
    next_nuoroda = f'https://www.aruodas.lt/{kitas_psl}'
    
    
    
